#### Imports & definitions

In [135]:
import json
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from collections import Counter
import plotly.io as pio
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

In [88]:
def read_json(path):
    with open(path) as file:
        return json.load(file)

    raise Exception("Failed to load json")

In [89]:
dataset_json = read_json('./data/dataset.json')

#### Best library stats

In [96]:
files_libraries_stats = {}

for data in dataset_json:
    file_name = data['file_name'].split('.json')[0]

    library_name = data['library_name']

    if file_name not in files_libraries_stats:
        files_libraries_stats[file_name] = {
            "NetworkX": 0,
            "Graph4J": 0,
            "JGraphT": 0
        }

    files_libraries_stats[file_name][library_name] += 1

files_libraries_arrays = {
    "NetworkX": [],
    "Graph4J": [],
    "JGraphT": []
}

files_libraries_majority = {
    "NetworkX": 0,
    "Graph4J": 0,
    "JGraphT": 0
}

for file_name in files_libraries_stats:
    files_libraries_arrays['NetworkX'].append(files_libraries_stats[file_name]['NetworkX'])
    files_libraries_arrays['Graph4J'].append(files_libraries_stats[file_name]['Graph4J'])
    files_libraries_arrays['JGraphT'].append(files_libraries_stats[file_name]['JGraphT'])

    files_libraries_majority[max(files_libraries_stats[file_name], key=files_libraries_stats[file_name].get)] += 1

y = list(files_libraries_stats.keys())

best_libraries_fig = go.Figure(
    data=[
        go.Bar(
            name='NetworkX',
            y=files_libraries_arrays['NetworkX'],
            x=y,
            #orientation='h'
        ),
        go.Bar(
            name='Graph4J',
            y=files_libraries_arrays['Graph4J'],
            x=y,
            #orientation='h'
        ),
        go.Bar(
            name='JGraphT',
            y=files_libraries_arrays['JGraphT'],
            x=y,
            #orientation='h'
        )
    ]
)

best_libraries_fig.update_layout(
    margin=dict(t=20, b=20),
    autosize=False,
    width=1200,
    height=500,
    yaxis_title="# times the library was the most performant",
    xaxis_title="Problem instance",
    legend_title="Library",
    font=dict(
        size=12,
    )
)

best_libraries_fig.update_yaxes(ticksuffix = " ")

pio.full_figure_for_development(best_libraries_fig, warn=False)
best_libraries_fig.write_image("output/best_libraries.eps")
              
best_libraries_fig.show()

#### Best algorithms stats

In [100]:
files_algorithms_stats = {}

for data in dataset_json:
    file_name = data['file_name'].split('.json')[0]

    algorithm_name = data['algorithm_name']

    if file_name not in files_algorithms_stats:
        files_algorithms_stats[file_name] = {
            "TabuCol": 0,
            "Coloring Genetic Algorithm": 0,
            "Coloring Backtrack": 0
        }

    files_algorithms_stats[file_name][algorithm_name] += 1

files_algorithms_arrays = {
    "TabuCol": [],
    "Coloring Genetic Algorithm": [],
    "Coloring Backtrack": []
}

files_algorithms_majority = {
    "TabuCol": 0,
    "Coloring Genetic Algorithm": 0,
    "Coloring Backtrack": 0
}

for file_name in files_libraries_stats:
    files_algorithms_arrays['TabuCol'].append(files_algorithms_stats[file_name]['TabuCol'])
    files_algorithms_arrays['Coloring Genetic Algorithm'].append(files_algorithms_stats[file_name]['Coloring Genetic Algorithm'])
    files_algorithms_arrays['Coloring Backtrack'].append(files_algorithms_stats[file_name]['Coloring Backtrack'])

    files_algorithms_majority[max(files_algorithms_stats[file_name], key=files_algorithms_stats[file_name].get)] += 1

y = list(files_algorithms_stats.keys())

best_algorithms_fig = go.Figure(
    data=[
        go.Bar(
            name='TabuCol',
            y=files_algorithms_arrays['TabuCol'],
            x=y,
            #orientation='h'
        ),
        go.Bar(
            name='Coloring Genetic Algorithm',
            y=files_algorithms_arrays['Coloring Genetic Algorithm'],
            x=y,
            #orientation='h'
        ),
        go.Bar(
            name='Coloring Backtrack',
            y=files_algorithms_arrays['Coloring Backtrack'],
            x=y,
            #orientation='h'
        )
    ]
)

best_algorithms_fig.update_layout(
    margin=dict(t=20, b=20),
    autosize=False,
    width=1200,
    height=500,
    yaxis_title="# times the algorithm was the most performant",
    xaxis_title="Problem instance",
    legend_title="Algorithm",
    font=dict(
        size=12,
    )
)

best_algorithms_fig.update_yaxes(ticksuffix = " ")

pio.full_figure_for_development(best_algorithms_fig, warn=False)
best_algorithms_fig.write_image("output/best_algorithms.eps")

best_algorithms_fig.show()

#### Majority stats

In [93]:
y = ['Library # files majority', 'Algorithm # files majority']

majority_fig = go.Figure(
    data=[
        go.Bar(
            name='TabuCol',
            y=[files_algorithms_majority['TabuCol']],
            x=['Algorithm']
        ),
        go.Bar(
            name='Coloring Genetic Algorithm',
            y=[files_algorithms_majority['Coloring Genetic Algorithm']],
            x=['Algorithm']
        ),
        go.Bar(
            name='Coloring Backtrack',
            y=[files_algorithms_majority['Coloring Backtrack']],
            x=['Algorithm']
        ),
        go.Bar(
            name='NetworkX',
            y=[files_libraries_majority['NetworkX']],
            x=['Library'],
        ),
        go.Bar(
            name='Graph4J',
            y=[files_libraries_majority['Graph4J']],
            x=['Library'],
        ),
        go.Bar(
            name='JGraphT',
            y=[files_libraries_majority['JGraphT']],
            x=['Library'],
        )
    ]
)

majority_fig.update_layout(
    margin=dict(t=20, b=20),
    autosize=False,
    width=800,
    height=400,
    xaxis_title="Algorithm/Library being the majority on a given file",
    yaxis_title="# problem instances",
    legend_title="Algorithm/Library",
    font=dict(
        size=12,
    )
)

majority_fig.update_yaxes(ticksuffix = " ")

pio.full_figure_for_development(majority_fig, warn=False)
majority_fig.write_image("output/majority.eps")
              
majority_fig.show()

In [176]:
file_names = list(files_algorithms_stats.keys())
files_features_majority = {}

for file_name in file_names:
    majority = max(files_algorithms_stats[file_name], key=files_algorithms_stats[file_name].get)
    features = None

    for data in dataset_json:
        if data['file_name'].split('.json')[0] == file_name:
            features = data['features']
            break

    files_features_majority[file_name] = {
        "features": features,
        "majority": majority
    }

features, colors = [], []

for file_name in files_features_majority:
    features.append(files_features_majority[file_name]['features'])

    if files_features_majority[file_name]['majority'] == 'TabuCol':
        colors.append('blue')
    elif files_features_majority[file_name]['majority'] == 'Coloring Genetic Algorithm':
        colors.append('red')
    elif files_features_majority[file_name]['majority'] == 'Coloring Backtrack':
        colors.append('green')

X = np.array(features)
colors = np.array(colors)

X_embedded = TSNE(n_components=3).fit_transform(X)
X_embedded.shape

(13, 3)

In [177]:
fig = go.Figure()

fig.add_trace(go.Scatter3d(x=X_embedded[colors=='blue', 0],
                           y=X_embedded[colors=='blue', 1],
                           z=X_embedded[colors=='blue', 2],
                           mode='markers',
                           name='TabuCol'))

fig.add_trace(go.Scatter3d(x=X_embedded[colors=='red', 0],
                           y=X_embedded[colors=='red', 1],
                           z=X_embedded[colors=='red', 2],
                           mode='markers',
                           name='Coloring Genetic Algorithm'))

fig.add_trace(go.Scatter3d(x=X_embedded[colors=='green', 0],
                           y=X_embedded[colors=='green', 1],
                           z=X_embedded[colors=='green', 2],
                           mode='markers',
                           name='Coloring Backtrack'))

fig.update_layout(
    title='T-SNE Mapping of the dataset',
    margin=dict(t=40, b=20),
    autosize=False,
    width=800,
    height=400,
)

fig.update_yaxes(ticksuffix = " ")

pio.full_figure_for_development(fig, warn=False)
fig.write_image("output/tsne_mapping.eps")

fig.show()